# EECS 595 Final Project
Zeyuan Li,   Wenbo Zhang,   Shuyan Li

## SETUP

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import shutil
import sys
assert sys.version_info[0]==3
assert sys.version_info[1] >= 5

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'EECS 595 NLP/final_project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)

print(os.listdir(GOOGLE_DRIVE_PATH))



['us_train.labels', 'us_train.text', 'us_test.labels', 'us_test.text', 'X_train_3D.torch', 'model_GLOVE_BiLSTM.pt', 'X_train_2D.torch', 'X_test_2D.torch', 'model_Emb_BiLSTM.pt', 'test_clean.pkl', 'train_clean.pkl', 'X_train_BERT.torch', 'X_train_mask_BERT.torch', 'X_test_BERT.torch', 'X_test_mask_BERT.torch', 'Y_test.torch', 'Y_train.torch', 'main.ipynb']


In [4]:
!pip install torchmetrics
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 30.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 25.7 MB/s 
     |████████████████████████████████| 182 kB 80.3 MB/s 
     |████████████████████████████████| 7.6 MB 67.1 MB/s 


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import bz2
import pandas as pd
import re 
import numpy as np
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from textblob import TextBlob
import string
import time

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

import gensim.downloader as api
from gensim.models import KeyedVectors
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchmetrics.classification import MulticlassF1Score

from transformers import BertModel
from transformers import BertTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## Data processing

### read data

In [ ]:
with open(GOOGLE_DRIVE_PATH + '/us_train.text', 'r') as f:
    train_txt = f.read().splitlines()
with open(GOOGLE_DRIVE_PATH + '/us_train.labels', 'r') as f:
    train_labels = f.read().splitlines()
with open(GOOGLE_DRIVE_PATH + '/us_test.text', 'r') as f:
    test_txt = f.read().splitlines()
with open(GOOGLE_DRIVE_PATH + '/us_test.labels', 'r') as f:
    test_labels = f.read().splitlines()


In [ ]:
train = pd.DataFrame()
train['text'] = train_txt
train['label'] = train_labels
train['label'] = pd.to_numeric(train['label'])

test = pd.DataFrame()
test['text'] = test_txt
test['label'] = test_labels
test['label'] = pd.to_numeric(test['label'])

In [ ]:
test

,text,label
0,Cheers Steven from Mickee and Happy Birthday-T...,5
1,Experiencing a heat wave in East Lansing @ Mic...,6
2,"️ ️ @ Southampton (village), New York",0
3,A few of my favorite things. @ Honey Pot,1
4,in love @ Disney's Magic Kingdom,8
...,...,...
49995,"These are the mail days I love @ Lexington, Ke...",5
49996,With special guest appearances by arigold03 an...,10
49997,Kishi. Bashi. ️ @ Sixth &amp; I,0
49998,Hangin' w/ @user in #hollywoodhills tonight wh...,6


### clean data

#### complex method

In [ ]:
def text_processor(X):
    '''
    input: string
    output: list of strings
    example:
        input = 'Lens faults: I purchased this lens to attach!'
        output = ['lens', 'fault', 'purchase', 'lens', 'attach']
    '''
    
    #---------------------------------------  step1 - lowercase ----------------------------------------------------#
    X = X.lower()
    
    #-----------------------  step2 - url, email, time, dollar amount, emoticon with special token -----------------#   
    ## 1.Times   ## 
    # 1.1 HH:MM:SS
    reg_time1 = "([0-1]?[0-9]|2[0-3]):[0-5][0-9]:[0-5][0-9]"
    # 1.2 HH:MM
    reg_time2 = "([0-1]?[0-9]|2[0-3]):[0-5][0-9]"
    X = re.sub(reg_time1, "time", X)
    X = re.sub(reg_time2, "time", X)
    
    ## 2.Dates  ##
    
    # 2.1 mm/dd/yyyy
    reg_date1 = "(0[1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])[- /.](19|20)\d\d"
    # 2.2 yyyy/mm/dd
    reg_date2 = "(19|20)\d\d[- /.](0[1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])"
    # 2.3 mm/yy/dd
    reg_date3 = "(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d"
    # 2.4 words like yesterday
    reg_date4 = "((t|T)oday)|((y|Y)esterday)|((t|T)omorrow)|((n|N)owadays)|((n|N)owadays)|(((next)|(last)|(Next)|(Last))\s((day)|(week)|(month)|(year)))"
    X = re.sub(reg_date1, "date", X)
    X = re.sub(reg_date2, "date", X)
    X = re.sub(reg_date3, "date", X)
    X = re.sub(reg_date4, "date", X)
    
    ## 3.Email addresses  ##
    
    reg_email1 = "[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+"
    X = re.sub(reg_email1, "email", X)
    
    ## 4.Web addXses  ##
    
    reg_web1 = "(http[s]?:\/\/(www\.)?|ftp:\/\/(www\.)?|www\.){1}([0-9A-Za-z-\.@:%_\+~#=]+)+((\.[a-zA-Z]{2,3})+)(/(.)*)?(\?(.)*)?"
    X = re.sub(reg_web1, "website", X)

    
    ## 5.Dollar amounts  ##

    # 5.1 $XX,XXX,XXX.XXXXX
    reg_dollar1 = "\$[0-9]+(,[0-9]{3})*\.[0-9]*"   
    # 5.2 $XX,XXX,XXX
    reg_dollar2 = "\$[0-9]+(,[0-9]{3})*"          
    X = re.sub(reg_dollar1, "dollar", X)
    X = re.sub(reg_dollar2, "dollar", X)
    
    ## emoticon
    reg_emoticon_happy = "(\:\))|(\;\))|(\:\-\))|(\:\-\])|(\:D)|(\:d)"
    X = re.sub(reg_emoticon_happy, "HAPPYEMOTICON", X)
    
    reg_emoticon_sad = "(\:\()|(\;\()|(\:\-\()|(\:\-\[)|(\:c)|(\:C)"   
    X = re.sub(reg_emoticon_sad, "SADEMOTICON", X)

   
    #------------------------------------  step3 - spelling correction -----------------------------------------------#
    textblob_ = TextBlob(X)
    X = textblob_.correct().string
    
    #------------------------------------  step4 - remove stopwords --------------------------------------------------#
    stopwords_english = stopwords.words('english')
    stopwords_nltk_en = set(stopwords_english)
    X = [word for word in X.split() if not word in stopwords_nltk_en]
    X = ' '.join(X)
    
    #------------------------------------  step5 - remove punctuations -----------------------------------------------#
    punc = string.punctuation
    X = X.translate(str.maketrans('', '', punc))
    
    #------------------------------------  step6 - remove digits -----------------------------------------------#
    X = re.sub(r'[0-9]+', '', X)
    
    #------------------------------------  step7- lemmatization -----------------------------------------------------#
    lemmatizer = WordNetLemmatizer()
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None

    pos_tagged = nltk.pos_tag(nltk.word_tokenize(X))
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    X = " ".join(lemmatized_sentence)

    #---------------------------------------  step8 - tokenization ----------------------------------------------------#
    X = word_tokenize(X)
    
    return X
    

try 100 samples

In [ ]:
train_part = train.iloc[:1000]
train_part['clean_text'] = train_part['text'].apply(lambda x: text_processor(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train_part

,text,label,clean_text
0,"LoL @ West Covina, California",2,"[west, come, california]"
1,Things got a little festive at the office #chr...,17,"[thing, get, little, festive, office, christma..."
2,Step out and explore. # ️ @ Ellis Island Cafe,0,"[step, explore, ️, elli, island, cafe]"
3,@user @ Cathedral Preparatory School,18,"[user, cathedral, preparatory, school]"
4,My baby bear @ Bubby's,1,"[baby, bear, bobby]"
...,...,...,...
995,@ Bitzer Park,17,"[bitter, park]"
996,I'm surrounded by beautiful friends ️ apprecia...,0,"[im, surround, beautiful, friend, ️, appreciat..."
997,I MADE MY BABIES! Yoshi is a black belt and be...,1,"[make, baby, tophi, black, belt, bear, hot, do..."
998,Thank you to everyone participated in Children...,8,"[thank, everyone, participate, childrens, grie..."


#### simple method

In [ ]:
def text_processor2(X):
    '''
    input: string
    output: list of strings
    example:
        input = 'Lens faults: I purchased this lens to attach!'
        output = ['lens', 'fault', 'purchase', 'lens', 'attach']
    '''
    #---------------------------------------  step1 - lowercase ----------------------------------------------------#
    X = X.lower()
    #print('lowercase------ ',X)

    #------------------------------------  step4 - remove stopwords --------------------------------------------------#
    stopwords_english = stopwords.words('english')
    stopwords_nltk_en = set(stopwords_english)
    X = [word for word in X.split() if not word in stopwords_nltk_en]
    X = ' '.join(X)
    #print('remove stopwords------ ',X)
    
    #------------------------------------  step5 - remove punctuations -----------------------------------------------#
    punc = string.punctuation
    X = X.translate(str.maketrans('', '', punc))
    #print('remove punctuations------ ',X)

    #------------------------------------  step6 - remove digits -----------------------------------------------#
    X = re.sub(r'[0-9]+', '', X)
    #print('remove digits------ ',X)

    #---------------------------------------  step7 - remove multiple spaces ----------------------------------------------------#
    X = re.sub(' +', ' ', X)
    #print('remove multiple spaces------ ',X)

    #---------------------------------------  step8 - tokenization ----------------------------------------------------#
    # X = word_tokenize(X)
    #print('tokenization------ ',X)
    return X

In [ ]:
train['clean_text_string'] = train['text'].apply(lambda x: text_processor2(x))
test['clean_text_string'] = test['text'].apply(lambda x: text_processor2(x))
train['clean_text'] = train['clean_text_string'].apply(lambda x: word_tokenize(x))
test['clean_text'] = test['clean_text_string'].apply(lambda x: word_tokenize(x))

In [ ]:
train.to_pickle(GOOGLE_DRIVE_PATH + "/train_clean.pkl")
test.to_pickle(GOOGLE_DRIVE_PATH + "/test_clean.pkl")

### load clean data

In [8]:
train = pd.read_pickle(GOOGLE_DRIVE_PATH + "/train_clean.pkl")
test = pd.read_pickle(GOOGLE_DRIVE_PATH + "/test_clean.pkl")

In [13]:
train

,text,label,clean_text_string,clean_text
0,"LoL @ West Covina, California",2,lol west covina california,"[lol, west, covina, california]"
1,Things got a little festive at the office #chr...,17,things got little festive office christmas red...,"[things, got, little, festive, office, christm..."
2,Step out and explore. # ️ @ Ellis Island Cafe,0,step explore ️ ellis island cafe,"[step, explore, ️, ellis, island, cafe]"
3,@user @ Cathedral Preparatory School,18,user cathedral preparatory school,"[user, cathedral, preparatory, school]"
4,My baby bear @ Bubby's,1,baby bear bubbys,"[baby, bear, bubbys]"
...,...,...,...,...
489604,Impromptu Sunday sunset picnic. #I ️SF #GGP @ ...,0,impromptu sunday sunset picnic i ️sf ggp golde...,"[impromptu, sunday, sunset, picnic, i, ️sf, gg..."
489605,I think we all miss blasé more than we miss ea...,3,think miss blasé miss probnot…,"[think, miss, blasé, miss, probnot…]"
489606,We voted! #election2016 #vote #proudtobeanamer...,11,voted election vote proudtobeanamerican usa br...,"[voted, election, vote, proudtobeanamerican, u..."
489607,"Retired Jerseys, where's Chris Webber's jersey...",19,retired jerseys wheres chris webbers jersey cr...,"[retired, jerseys, wheres, chris, webbers, jer..."


## model0 - baseline: Naive Bayes

### tf-idf

In [14]:
vectorizer = TfidfVectorizer(min_df = 1000)
X_train_vec = vectorizer.fit_transform(train['clean_text_string'])
Y_train_vec = np.array(train['label'])
X_test_vec = vectorizer.transform(test['clean_text_string'])
Y_test_vec = np.array(test['label'])


### NB

In [18]:
NB_model = MultinomialNB()

NB_model.fit(X_train_vec, Y_train_vec)

ACC_train = NB_model.score(X_train_vec, Y_train_vec)
ACC_test = NB_model.score(X_test_vec, Y_test_vec)
print("train accuracy: %.2f%%, test accuracy: %.2f%%" % (ACC_train * 100.0, ACC_test * 100.0))

train accuracy: 27.55%, test accuracy: 27.25%


## model1 - GloVe + BiLSTM

### process data to 3D tensor

Load word-to-vector model

In [ ]:
# GloVe = api.load('glove-wiki-gigaword-100')
GloVe = api.load("glove-twitter-100")
# GloVe.get_vector('11.2')
# GloVe_vocab = GloVe.vocab.keys()

In [ ]:
#train = train_part
DEVICE = 'cuda:0'

# get max length of sentences
max_length = 0
for sentence in train['clean_text']:
  max_length = max(max_length, len(sentence))


# train
N = len(train)
X_train = torch.zeros(N, max_length, 100, device = DEVICE)
Y_train = torch.tensor(train['label'], device = DEVICE)
for i, sentence in enumerate(train['clean_text']):
    this_matrix = torch.zeros(max_length, 100, device = DEVICE)
    for j, word in enumerate(sentence):
        try:
            this_matrix[j] = torch.tensor(GloVe.get_vector(word), device = DEVICE)
        except:
            pass
    X_train[i] = this_matrix

# test
N = len(test)
X_test = torch.zeros(N, max_length, 100, device = DEVICE)
Y_test = torch.tensor(test['label'], device = DEVICE)
for i, sentence in enumerate(test['clean_text']):
    this_matrix = torch.zeros(max_length, 100, device = DEVICE)
    for j, word in enumerate(sentence):
        try:
            this_matrix[j] = torch.tensor(GloVe.get_vector(word), device = DEVICE)
        except:
            pass
    X_test[i] = this_matrix
 

Don't run saving here, the kernal will shut down!!!!!

In [ ]:
torch.save(X_train, GOOGLE_DRIVE_PATH + '/X_train_3D.torch')
torch.save(Y_train, GOOGLE_DRIVE_PATH + '/Y_train.torch')

torch.save(X_test, GOOGLE_DRIVE_PATH + '/X_test_3D.torch')
torch.save(Y_test, GOOGLE_DRIVE_PATH + '/Y_test.torch')

### load X_train and Y_train 
- the file may not exist, because saving step failed

In [ ]:
X_train = torch.load(GOOGLE_DRIVE_PATH + '/X_train_3D.torch')
Y_train = torch.load(GOOGLE_DRIVE_PATH + '/Y_train.torch')
X_test = torch.load(GOOGLE_DRIVE_PATH + '/X_test_3D.torch')
Y_test = torch.load(GOOGLE_DRIVE_PATH + '/Y_test.torch')

### define model

define model
- we keep two vectors, one for the forward (`hidden[-1, :, :]` will take the last row) and one for the backward networks (`hidden[-2, :, :]` will take the second row from the end).

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, sentence_length, embedding_dim, hidden_dim1, hidden_dim2, tagset_size, num_layers, dropout):
        super().__init__()

        self.lstm = nn.LSTM( embedding_dim, 
                    hidden_dim1, 
                    num_layers = num_layers,
                    #dropout = dropout,
                    bidirectional = True,
                    batch_first = True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p = dropout)
        self.fc1 = nn.Linear(2 * hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p = dropout)
        self.fc2 = nn.Linear(hidden_dim2, tagset_size)

    def forward(self, sentence):
        packed_output, (hidden, cell) = self.lstm(sentence)
        cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.relu1(cat)
        out = self.dropout1(out)
        out = self.fc1(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        score = self.fc2(out)
        return score

In [ ]:
def train_model(X_train, Y_train, model, sentence_length, embedding_dim, hidden_dim1, hidden_dim2,
          tagset_size, num_layers, dropout, batch_size, epochs, lr, weight_decay, 
          gradient_clipping, seed, device, X_test=None, Y_test=None):
  torch.manual_seed(seed)

  # model definition
  if model == 'BiLSTM':
    model = BiLSTM(sentence_length, embedding_dim, hidden_dim1, hidden_dim2, tagset_size, num_layers, dropout)
    model = model.to(DEVICE)
    optomizer = torch.optim.Adam(model.parameters(),lr = lr, weight_decay = weight_decay)
    loss_func = nn.CrossEntropyLoss(ignore_index=0)


  # model fitting
  print('-------------- training model -----------------')
  print('Device:', DEVICE)
  for epoch in range(epochs):
      torch.manual_seed(seed)
      np.random.seed(seed)
      #  get batch data
      batch_data = DataLoader(list(zip(X_train, Y_train)), batch_size = batch_size, shuffle=True)
      for X_batch, Y_batch in batch_data:
          model.train()
          optomizer.zero_grad()
          logit = model.forward(X_batch)
          # print(logit.shape)
          # print(Y_batch.shape)
          loss = loss_func(logit, Y_batch.reshape(-1).long())
          loss.backward()
          optomizer.step()

      model.eval()
      train_acc, train_f1, _ = test_model(model, X_train, Y_train)
      # compute test loss for each epoch, batch by batch
      if X_test is not None:
        # batch_data = DataLoader(list(zip(X_test, Y_test)), batch_size = batch_size, shuffle=True)
        # metric = evaluate.load("accuracy")
        # model.eval()
        # loss_func_test = nn.CrossEntropyLoss(reduce='mean')
        # loss = 0
        # with torch.no_grad():
        #     logit = model(X_test)
        # loss = loss_func_test(logit, Y_test.reshape(-1).long())
        # predictions = torch.argmax(logit, dim=-1)
        # metric.add_batch(predictions=predictions, references=Y_test)
        # score = metric.compute()
        # print('Test Loss:{}, Test Accuracy:{}'.format(loss, score['accuracy']))
        model.eval()
        test_acc, test_f1, _ = test_model(model, X_test, Y_test)
        print('epoch:{}/{}, train loss:{}, train acc:{}, train F1:{}, test acc:{}, test F1:{}'.format(epoch, epochs, loss, train_acc,train_f1, test_acc,test_f1))
      else:
        print('epoch:{}/{}, train loss:{}, train acc:{}'.format(epoch, epochs, loss, train_acc))
  print('-------------- training model finished-----------------')
  return model 

def test_model(model, X_test, Y_test, batch_size=1000):
  metric = MulticlassF1Score(num_classes = 20)
  batch_data = DataLoader(list(zip(X_test, Y_test)), batch_size = batch_size, shuffle=True)
  correct_count = 0
  Y_pred_all = []
  for X_batch, Y_batch in batch_data:
    model.eval()
    Y_pred = model.forward(X_batch)
    Y_pred = torch.argmax(Y_pred, dim=1)
    correct_count += np.sum(np.array(Y_pred.cpu() == Y_batch.cpu()))
    Y_pred_all = Y_pred_all + Y_pred.tolist()
  acc = correct_count / X_test.shape[0]
  Y_pred_all = torch.tensor(Y_pred_all)
  F1_score = metric(Y_pred_all.to('cpu'), Y_test.to('cpu'))
  return acc, F1_score, Y_pred_all

### train

In [ ]:
params = {'model': 'BiLSTM',
    'sentence_length': X_train.shape[1],
    'embedding_dim': X_train.shape[2],
    'hidden_dim1': 128,
    'hidden_dim2': 64,
    'tagset_size': 20,
    'num_layers': 2,
    'dropout': 0.5,
    'batch_size': 128,
    'epochs': 30,
    'lr': 0.001,
    'weight_decay': 0,
    'gradient_clipping': 1000,
    'seed': 0,
    'device': DEVICE,
    'X_test': X_test,
    'Y_test': Y_test
}
model = train_model(X_train, Y_train, **params)

-------------- training model -----------------
Device: cuda:0
epoch:0/30, train loss:2.647008180618286, train acc:0.25116368367411546, train F1:0.030450139194726944, test acc:0.24888, test F1:0.03024796023964882
epoch:1/30, train loss:2.1864891052246094, train acc:0.26470101652543154, train F1:0.03269466385245323, test acc:0.26386, test F1:0.031733568757772446
epoch:2/30, train loss:1.9530080556869507, train acc:0.27405950462511924, train F1:0.033490486443042755, test acc:0.27212, test F1:0.032065294682979584
epoch:3/30, train loss:1.7460919618606567, train acc:0.28149400848432116, train F1:0.03389430046081543, test acc:0.27752, test F1:0.0321553610265255
epoch:4/30, train loss:1.5543346405029297, train acc:0.2873844230804581, train F1:0.03433427959680557, test acc:0.2803, test F1:0.03236769139766693
epoch:5/30, train loss:1.328529715538025, train acc:0.2920064786390773, train F1:0.034449879080057144, test acc:0.28262, test F1:0.0331796258687973
epoch:6/30, train loss:1.21533584594726

save model

In [ ]:
torch.save(model, GOOGLE_DRIVE_PATH + '/model_GLOVE_BiLSTM.pt')

load model

In [ ]:
model = torch.load(GOOGLE_DRIVE_PATH + '/model_GLOVE_BiLSTM.pt')

accuracy and F1

In [ ]:
acc1, f1_1, y_pred1 = test(model, X_train, Y_train)
acc2, f1_2, y_pred2 = test(model, X_test, Y_test)
print('train accuracy:',acc1, 'test accuracy:', acc2)

train accuracy: 0.3266790438901246 test accuracy: 0.29414


## model2 - embedding layer + BiLSTM

#### create vocab_to_idx. 
- If occurance of word <=3, then make it to be token 'UNKA'

In [ ]:
count = {}

for words in train['clean_text']:
  for word in words:
    count[word] = 1 + count.get(word, 0)

for i,sentence in enumerate(train['clean_text']):
    for j, word in enumerate(sentence):
        if count[word] < 3:
            train['clean_text'][i][j] = 'UNKA'


vocab_to_idx = {'UNKA': 1}
i = 2
for words in train['clean_text']:
  for word in words:
    if word not in vocab_to_idx:
      vocab_to_idx[word] = i
      i += 1
      if i % 10000 == 0:
        print(i)

10000
20000
30000
40000
50000


### process data to 2D tensor: N * L
- L is the max length of all sentences

In [ ]:
DEVICE = 'cuda:0'

# get max length of sentences
max_length = 0
for sentence in train['clean_text']:
  max_length = max(max_length, len(sentence))


# train
N = len(train)
X_train = torch.zeros(N, max_length, device = DEVICE)
Y_train = torch.tensor(train['label'], device = DEVICE)
for i, sentence in enumerate(train['clean_text']):
    this_array = torch.zeros(max_length, device = DEVICE)
    for j, word in enumerate(sentence):
      this_array[j] = torch.tensor(vocab_to_idx[word], device = DEVICE)
    X_train[i] = this_array

# test 
N = len(test)
X_test = torch.zeros(N, max_length, device = DEVICE)
Y_test = torch.tensor(test['label'], device = DEVICE)
for i, sentence in enumerate(test['clean_text']):
    this_array = torch.zeros(max_length, device = DEVICE)
    for j, word in enumerate(sentence):
      if word in vocab_to_idx:
        this_array[j] = torch.tensor(vocab_to_idx[word], device = DEVICE)
      else:
        this_array[j] = torch.tensor(vocab_to_idx['UNKA'], device = DEVICE)
    X_test[i] = this_array



In [ ]:
X_train = X_train.to(torch.int32)
torch.save(X_train, GOOGLE_DRIVE_PATH + '/X_train_2D.torch')
torch.save(Y_train, GOOGLE_DRIVE_PATH + '/Y_train.torch')

X_test = X_test.to(torch.int32)
torch.save(X_test, GOOGLE_DRIVE_PATH + '/X_test_2D.torch')
torch.save(Y_test, GOOGLE_DRIVE_PATH + '/Y_test.torch')

### load X_train and Y_train

In [ ]:
X_train = torch.load(GOOGLE_DRIVE_PATH + '/X_train_2D.torch')
Y_train = torch.load(GOOGLE_DRIVE_PATH + '/Y_train.torch')
X_test = torch.load(GOOGLE_DRIVE_PATH + '/X_test_2D.torch')
Y_test = torch.load(GOOGLE_DRIVE_PATH + '/Y_test.torch')

In [ ]:
DEVICE = 'cuda:0'
X_train = X_train.to(DEVICE)
Y_train = Y_train.to(DEVICE)
X_test = X_test.to(DEVICE)
Y_test = Y_test.to(DEVICE)

make fake test set: sample 40000 of train + 10000 of test

In [ ]:
# idx = np.arange(X_train.shape[0])
# val_idx = np.random.choice(idx, 40000)
# X_temp1 = X_train[val_idx]
# Y_temp1= Y_train[val_idx]

# idx = np.arange(X_test.shape[0])
# val_idx = np.random.choice(idx, 10000)
# X_temp2 = X_test[val_idx]
# Y_temp2= Y_test[val_idx]

# X_val = torch.cat((X_temp1, X_temp2), 0)
# Y_val = torch.cat((Y_temp1, Y_temp2), 0)

### define model

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, sentence_length, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, tagset_size, num_layers, dropout):
        super().__init__()
        #self.sentence_length = torch.tensor(sentence_length)

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = 0)
        self.lstm = nn.LSTM( embedding_dim, 
                    hidden_dim1, 
                    num_layers = num_layers,
                    #dropout = dropout,
                    bidirectional = True,
                    batch_first = True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p = dropout)
        self.fc1 = nn.Linear(2 * hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p = dropout)
        self.fc2 = nn.Linear(hidden_dim2, tagset_size)

    def forward(self, sentence):
        
        embedded = self.embedding(sentence)
        # print(embedded.shape)
        # packed_embedded = pack_padded_sequence(embedded, self.sentence_length, batch_first=True) 
        packed_output, (hidden, cell) = self.lstm(embedded)
        cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.relu1(cat)
        #print('lstm_out', lstm_out.shape)
        # B, L, H = out.shape
        # out = out.reshape(B, L*H)
        #out = self.dropout1(out)
        out = self.fc1(out)
        #out = self.relu2(out)
        out = self.dropout2(out)
        score = self.fc2(out)
        return score

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, sentence_length, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, tagset_size, num_layers, dropout):
        super().__init__()
        #self.sentence_length = torch.tensor(sentence_length)

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = 0)
        self.lstm = nn.LSTM( embedding_dim, 
                    hidden_dim1, 
                    num_layers = num_layers,
                    #dropout = dropout,
                    bidirectional = True,
                    batch_first = True)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p = dropout)
        self.fc1 = nn.Linear(2 * hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p = dropout)
        self.fc2 = nn.Linear(hidden_dim2, tagset_size)

    def forward(self, sentence):
        
        embedded = self.embedding(sentence)
        # print(embedded.shape)
        # packed_embedded = pack_padded_sequence(embedded, self.sentence_length, batch_first=True) 
        packed_output, (hidden, cell) = self.lstm(embedded)
        cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.relu1(cat)
        out = self.dropout1(out)
        out = self.fc1(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        score = self.fc2(out)
        return score

In [ ]:
def train_model(X_train, Y_train, model, sentence_length, vocab_size, embedding_dim, hidden_dim1, hidden_dim2,
          tagset_size, num_layers, dropout, batch_size, epochs, lr, weight_decay, 
          gradient_clipping, seed, device, X_test=None, Y_test=None):
  torch.manual_seed(seed)

  # model definition
  if model == 'BiLSTM':
    model = BiLSTM(sentence_length, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, tagset_size, num_layers, dropout)
    model = model.to(DEVICE)
    optomizer = torch.optim.Adam(model.parameters(),lr = lr, weight_decay = weight_decay)
    loss_func = nn.CrossEntropyLoss(ignore_index=0)
  elif model == 'textCNN':
    pass
  elif model == 'BERT':
    pass

  # model fitting
  print('-------------- training model -----------------')
  print('Device:', DEVICE)
  for epoch in range(epochs):
      torch.manual_seed(seed)
      np.random.seed(seed)
      #  get batch data
      batch_data = DataLoader(list(zip(X_train, Y_train)), batch_size = batch_size, shuffle=True)
      for X_batch, Y_batch in batch_data:
          model.train()
          optomizer.zero_grad()
          logit = model.forward(X_batch)
          # print(logit.shape)
          # print(Y_batch.shape)
          loss = loss_func(logit, Y_batch.reshape(-1).long())
          loss.backward()
          optomizer.step()

      model.eval()
      train_acc, train_f1, _ = test_model(model, X_train, Y_train)
      # compute test loss for each epoch, batch by batch
      if X_test is not None:
        # batch_data = DataLoader(list(zip(X_test, Y_test)), batch_size = batch_size, shuffle=True)
        # metric = evaluate.load("accuracy")
        # model.eval()
        # loss_func_test = nn.CrossEntropyLoss(reduce='mean')
        # loss = 0
        # with torch.no_grad():
        #     logit = model(X_test)
        # loss = loss_func_test(logit, Y_test.reshape(-1).long())
        # predictions = torch.argmax(logit, dim=-1)
        # metric.add_batch(predictions=predictions, references=Y_test)
        # score = metric.compute()
        # print('Test Loss:{}, Test Accuracy:{}'.format(loss, score['accuracy']))
        model.eval()
        test_acc, test_f1, _ = test_model(model, X_test, Y_test)
        print('epoch:{}/{}, train loss:{}, train acc:{}, train F1:{}, test acc:{}, test F1:{}'.format(epoch, epochs, loss, train_acc,train_f1, test_acc,test_f1))
      else:
        print('epoch:{}/{}, train loss:{}, train acc:{}'.format(epoch, epochs, loss, train_acc))
  print('-------------- training model finished-----------------')
  return model 

def test_model(model, X_test, Y_test, batch_size=1000):
  metric = MulticlassF1Score(num_classes = 20)
  batch_data = DataLoader(list(zip(X_test, Y_test)), batch_size = batch_size, shuffle=True)
  correct_count = 0
  Y_pred_all = []
  for X_batch, Y_batch in batch_data:
    model.eval()
    Y_pred = model.forward(X_batch)
    Y_pred = torch.argmax(Y_pred, dim=1)
    correct_count += np.sum(np.array(Y_pred.cpu() == Y_batch.cpu()))
    Y_pred_all = Y_pred_all + Y_pred.tolist()
  acc = correct_count / X_test.shape[0]
  Y_pred_all = torch.tensor(Y_pred_all)
  F1_score = metric(Y_pred_all.to('cpu'), Y_test.to('cpu'))
  return acc, F1_score, Y_pred_all

### train

In [ ]:
params = {'model': 'BiLSTM',
    'sentence_length': X_train.shape[1],
    #'vocab_size': len(vocab_to_idx)+1,
    'vocab_size': 58490,
    'embedding_dim': 100,
    'hidden_dim1': 128,
    'hidden_dim2': 64,
    'tagset_size': 20,
    'num_layers': 2,
    'dropout': 0.5,
    'batch_size': 128,
    'epochs': 30,
    'lr': 0.001,
    'weight_decay': 0,
    'gradient_clipping': 1000,
    'seed': 0,
    'device': DEVICE,
    'X_test': X_test,
    'Y_test': Y_test
}
model = train_model(X_train, Y_train, **params)

-------------- training model -----------------
Device: cuda:0
epoch:0/30, train loss:2.824125051498413, train acc:0.25353496361382244, train F1:0.028904173523187637, test acc:0.24914, test F1:0.02808164805173874
epoch:1/30, train loss:2.2442729473114014, train acc:0.2793004213566336, train F1:0.030505405738949776, test acc:0.27248, test F1:0.030429981648921967
epoch:2/30, train loss:1.7164872884750366, train acc:0.3003049372049942, train F1:0.032392941415309906, test acc:0.28566, test F1:0.032470569014549255
epoch:3/30, train loss:1.4365568161010742, train acc:0.31884422059235024, train F1:0.0334390327334404, test acc:0.29718, test F1:0.0329466238617897
epoch:4/30, train loss:1.2427504062652588, train acc:0.3366318838093254, train F1:0.034760065376758575, test acc:0.30648, test F1:0.03396362066268921
epoch:5/30, train loss:1.056728482246399, train acc:0.352979622515109, train F1:0.035452235490083694, test acc:0.31532, test F1:0.034223876893520355
epoch:6/30, train loss:0.9745862483978

save model

In [ ]:
torch.save(model, GOOGLE_DRIVE_PATH + '/model_Emb_BiLSTM.pt')

load model

In [ ]:
model = torch.load(GOOGLE_DRIVE_PATH + '/model_Emb_BiLSTM.pt')

accuracy

In [ ]:
acc1, f1_1, y_pred1 = test(model, X_train, Y_train)
acc2, f1_2, y_pred2 = test(model, X_val, Y_val)
print('train accuracy:',acc1, 'test accuracy:', acc2)


train accuracy: 0.49007677554946905 test accuracy: 0.43536


In [ ]:
(X_train == 58488).nonzero(as_tuple=True)

(tensor([475894, 479845, 485409]), tensor([2, 8, 2]))

## model3 - BERT

### BERT tokenizer
- use pre-trained BERT tokenizer to turn cleaned string into tensor of ids

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# train_bert_input = tokenizer(list(train['clean_text_string']), padding='max_length', truncation=True, return_tensors="pt")
# test_bert_input = tokenizer(list(test['clean_text_string']), padding='max_length', truncation=True, return_tensors="pt")
DEVICE = 'cpu'

# get max length of sentences
max_length = 0
for sentence in train['clean_text']:
  max_length = max(max_length, len(sentence))


# train
N = len(train)
# BERT will add 2 tokens [SEP] and [END] to sentence, so max_length+2
X_train = torch.zeros(N, max_length+2, device = DEVICE)
X_train_mask = torch.zeros(N, max_length+2, device = DEVICE)
Y_train = torch.tensor(train['label'], device = DEVICE)
for i, sentence in enumerate(train['clean_text_string']):
    train_bert_input = tokenizer(sentence, padding='max_length', max_length = max_length+2, truncation=True, return_tensors="pt")
    X_train[i] = train_bert_input['input_ids'].to(DEVICE)
    X_train_mask[i] = train_bert_input['attention_mask'].to(DEVICE)
    if i % 10000 == 0:
      print(i)

# test 
N = len(test)
X_test = torch.zeros(N, max_length+2, device = DEVICE)
X_test_mask = torch.zeros(N, max_length+2, device = DEVICE)
Y_test = torch.tensor(test['label'], device = DEVICE)
for i, sentence in enumerate(test['clean_text_string']):
    test_bert_input = tokenizer(sentence, padding='max_length', max_length = max_length+2, truncation=True, return_tensors="pt")
    X_test[i] = test_bert_input['input_ids'].to(DEVICE)
    X_test_mask[i] = test_bert_input['attention_mask'].to(DEVICE)
    if i % 10000 == 0:
      print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
0
10000
20000
30000
40000


In [ ]:
torch.save(X_train, GOOGLE_DRIVE_PATH + '/X_train_BERT.torch')
torch.save(X_train_mask, GOOGLE_DRIVE_PATH + '/X_train_mask_BERT.torch')
torch.save(Y_train, GOOGLE_DRIVE_PATH + '/Y_train.torch')

torch.save(X_test, GOOGLE_DRIVE_PATH + '/X_test_BERT.torch')
torch.save(X_test_mask, GOOGLE_DRIVE_PATH + '/X_test_mask_BERT.torch')
torch.save(Y_test, GOOGLE_DRIVE_PATH + '/Y_test.torch')

### load X_train

In [ ]:
X_train = torch.load(GOOGLE_DRIVE_PATH + '/X_train_BERT.torch')
X_train_mask = torch.load(GOOGLE_DRIVE_PATH + '/X_train_mask_BERT.torch')
Y_train = torch.load(GOOGLE_DRIVE_PATH + '/Y_train.torch')
X_test = torch.load(GOOGLE_DRIVE_PATH + '/X_test_BERT.torch')
X_test_mask = torch.load(GOOGLE_DRIVE_PATH + '/X_test_mask_BERT.torch')
Y_test = torch.load(GOOGLE_DRIVE_PATH + '/Y_test.torch')

DEVICE = 'cpu'
X_train = X_train.to(torch.int32)
X_train = X_train.to(DEVICE)
X_train_mask = X_train_mask.to(DEVICE)
Y_train = Y_train.to(DEVICE)
X_test = X_test.to(torch.int32)
X_test = X_test.to(DEVICE)
X_test_mask = X_test_mask.to(DEVICE)
Y_test = Y_test.to(DEVICE)

In [ ]:
X_train

tensor([[  101, 25338,  1233,  ...,     0,     0,     0],
        [  101,  1614,  1400,  ...,     0,     0,     0],
        [  101,  2585,  8664,  ...,     0,     0,     0],
        ...,
        [  101,  4751,  1728,  ...,     0,     0,     0],
        [  101,  2623, 14953,  ...,     0,     0,     0],
        [  101,  6159,  7076,  ...,     0,     0,     0]], device='cuda:0',
       dtype=torch.int32)

### define model

In [ ]:
class BERT(nn.Module):

    def __init__(self, hidden_dim, target_size, dropout=0.5):

        super(BERT, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(hidden_dim, target_size)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        out = self.dropout(pooled_output)
        out = self.linear(out)
        #out = self.relu(out)

        return out

In [ ]:
def train_model(X_train, X_train_mask, Y_train, hidden_dim, target_size, dropout, batch_size, epochs, lr, weight_decay, 
          gradient_clipping, seed, device, X_test=None, Y_test=None):
  
  torch.manual_seed(seed)
  model = BERT(hidden_dim, target_size, dropout)
  model = model.to(DEVICE)
  optomizer = torch.optim.Adam(model.parameters(),lr = lr, weight_decay = weight_decay)
  loss_func = nn.CrossEntropyLoss()
  loss_func = loss_func.to(DEVICE)
  # model fitting
  print('-------------- training model -----------------')
  print('Device:', DEVICE)
  for epoch in range(epochs):
      torch.manual_seed(seed)
      np.random.seed(seed)
      #  get batch data
      batch_data = DataLoader(list(zip(X_train, X_train_mask, Y_train)), batch_size = batch_size, shuffle=True)
      for X_batch, X_mask_batch, Y_batch in batch_data:
          model.train()
          optomizer.zero_grad()
          logit = model.forward(X_batch, X_mask_batch)
          loss = loss_func(logit, Y_batch.reshape(-1).long())
          print(loss)
          loss.backward()
          optomizer.step()

      model.eval()
      train_acc, train_f1, _ = test_model(model, X_train, X_train_mask, Y_train)
      # compute test loss for each epoch, batch by batch
      if X_test is not None:
        model.eval()
        test_acc, test_f1, _ = test_model(model, X_test, X_test_mask, Y_test)
        print('epoch:{}/{}, train loss:{}, train acc:{}, train F1:{}, test acc:{}, test F1:{}'.format(epoch, epochs, loss, train_acc,train_f1, test_acc,test_f1))
      else:
        print('epoch:{}/{}, train loss:{}, train acc:{}'.format(epoch, epochs, loss, train_acc))
  print('-------------- training model finished-----------------')
  return model 

def test_model(model, X_test, X_test_mask, Y_test, batch_size=1000):
  metric = MulticlassF1Score(num_classes = 20)
  batch_data = DataLoader(list(zip(X_test, X_test_mask, Y_test)), batch_size = batch_size, shuffle=True)
  correct_count = 0
  Y_pred_all = []
  for X_batch, X_mask_batch, Y_batch in batch_data:
    model.eval()
    Y_pred = model.forward(X_batch, X_mask_batch)
    Y_pred = torch.argmax(Y_pred, dim=1)
    correct_count += np.sum(np.array(Y_pred.cpu() == Y_batch.cpu()))
    Y_pred_all = Y_pred_all + Y_pred.tolist()
  acc = correct_count / X_test.shape[0]
  Y_pred_all = torch.tensor(Y_pred_all)
  F1_score = metric(Y_pred_all.to('cpu'), Y_test.to('cpu'))
  return acc, F1_score, Y_pred_all

### train

In [ ]:
params = {
    'hidden_dim': 768,
    'target_size': 20,
    'dropout': 0.5,
    'batch_size': 128,
    'epochs': 30,
    'lr': 0.0001,
    'weight_decay': 0,
    'gradient_clipping': 1000,
    'seed': 0,
    'device': DEVICE,
    'X_test': X_test,
    'Y_test': Y_test
}
model = train_model(X_train[:10], X_train_mask[:10], Y_train[:10], **params)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


-------------- training model -----------------
Device: cpu
tensor(3.2027, grad_fn=<NllLossBackward0>)


***Attention: BERT needs to be trained on Great Lakes***

## model 4 - text CNN

### load X_train
- the same as the input of model 2 - embedding layer + BiLSTM

In [13]:
X_train = torch.load(GOOGLE_DRIVE_PATH + '/X_train_2D.torch')
Y_train = torch.load(GOOGLE_DRIVE_PATH + '/Y_train.torch')
X_test = torch.load(GOOGLE_DRIVE_PATH + '/X_test_2D.torch')
Y_test = torch.load(GOOGLE_DRIVE_PATH + '/Y_test.torch')

DEVICE = 'cuda:0'
X_train = X_train.to(torch.int32)
X_train = X_train.to(DEVICE)
X_train_mask = X_train_mask.to(DEVICE)
Y_train = Y_train.to(DEVICE)
X_test = X_test.to(torch.int32)
X_test = X_test.to(DEVICE)
X_test_mask = X_test_mask.to(DEVICE)
Y_test = Y_test.to(DEVICE)

### define model

In [22]:
class textCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, target_size, dropout, hidden_size, pool_size, filter_sizes):
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels = 1,
                             out_channels = n_filters,
                             kernel_size = (fs, embedding_dim)) 
                        for fs in filter_sizes])
        self.max_pool1 = nn.MaxPool1d(pool_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(33*n_filters, hidden_size, bias=True)  
        self.fc2 = nn.Linear(hidden_size, target_size, bias=True)  

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)      
        convolution = [conv(embedded) for conv in self.convs]   
        max1 = self.max_pool1(convolution[0].squeeze()) 
        max2 = self.max_pool1(convolution[1].squeeze())
        #print('max1', max1.shape)
        #print('max2', max2.shape)
        cat = torch.cat((max1, max2), dim=2)
        #print('cat', cat.shape)      
        x = cat.view(cat.shape[0], -1) 
        x = self.fc1(self.relu(x))
        x = self.dropout(x)
        x = self.fc2(x)  
        return x

In [20]:
def train_model(X_train, Y_train, model, vocab_size, embedding_dim, n_filters,
          tagset_size, dropout, hidden_size, pool_size, filter_sizes
          ,batch_size, epochs, lr, weight_decay, 
          gradient_clipping, seed, device, X_test=None, Y_test=None):
  torch.manual_seed(seed)

  # model definition
  model = textCNN(vocab_size, embedding_dim, n_filters, tagset_size, dropout, hidden_size, pool_size, filter_sizes)
  model = model.to(DEVICE)
  optomizer = torch.optim.Adam(model.parameters(),lr = lr, weight_decay = weight_decay)
  loss_func = nn.CrossEntropyLoss()

  # model fitting
  print('-------------- training model -----------------')
  print('Device:', DEVICE)
  for epoch in range(epochs):
      torch.manual_seed(seed)
      np.random.seed(seed)
      #  get batch data
      batch_data = DataLoader(list(zip(X_train, Y_train)), batch_size = batch_size, shuffle=True)
      for X_batch, Y_batch in batch_data:
          model.train()
          optomizer.zero_grad()
          logit = model.forward(X_batch)
          # print(logit.shape)
          # print(Y_batch.shape)
          loss = loss_func(logit, Y_batch.reshape(-1).long())
          loss.backward()
          optomizer.step()

      model.eval()
      train_acc, train_f1, _ = test_model(model, X_train, Y_train)
      # compute test loss for each epoch, batch by batch
      if X_test is not None:
        # batch_data = DataLoader(list(zip(X_test, Y_test)), batch_size = batch_size, shuffle=True)
        # metric = evaluate.load("accuracy")
        # model.eval()
        # loss_func_test = nn.CrossEntropyLoss(reduce='mean')
        # loss = 0
        # with torch.no_grad():
        #     logit = model(X_test)
        # loss = loss_func_test(logit, Y_test.reshape(-1).long())
        # predictions = torch.argmax(logit, dim=-1)
        # metric.add_batch(predictions=predictions, references=Y_test)
        # score = metric.compute()
        # print('Test Loss:{}, Test Accuracy:{}'.format(loss, score['accuracy']))
        model.eval()
        test_acc, test_f1, _ = test_model(model, X_test, Y_test)
        print('epoch:{}/{}, train loss:{}, train acc:{}, train F1:{}, test acc:{}, test F1:{}'.format(epoch, epochs, loss, train_acc,train_f1, test_acc,test_f1))
      else:
        print('epoch:{}/{}, train loss:{}, train acc:{}'.format(epoch, epochs, loss, train_acc))
  print('-------------- training model finished-----------------')
  return model 

def test_model(model, X_test, Y_test, batch_size=1000):
  metric = MulticlassF1Score(num_classes = 20)
  batch_data = DataLoader(list(zip(X_test, Y_test)), batch_size = batch_size, shuffle=True)
  correct_count = 0
  Y_pred_all = []
  for X_batch, Y_batch in batch_data:
    model.eval()
    Y_pred = model.forward(X_batch)
    Y_pred = torch.argmax(Y_pred, dim=1)
    correct_count += np.sum(np.array(Y_pred.cpu() == Y_batch.cpu()))
    Y_pred_all = Y_pred_all + Y_pred.tolist()
  acc = correct_count / X_test.shape[0]
  Y_pred_all = torch.tensor(Y_pred_all)
  F1_score = metric(Y_pred_all.to('cpu'), Y_test.to('cpu'))
  return acc, F1_score, Y_pred_all

### train

In [27]:
params = {'model': 'textCNN',
    'vocab_size': 58490,
    'embedding_dim': 100,
    'n_filters': 128,
    'tagset_size': 20,
    'dropout': 0.5,
    'hidden_size': 128,
    'pool_size': 2,
    'filter_sizes': [3, 8],
    'batch_size': 128,
    'epochs': 5,
    'lr': 0.001,
    'weight_decay': 0,
    'gradient_clipping': 1000,
    'seed': 0,
    'device': DEVICE,
    'X_test': X_test,
    'Y_test': Y_test
}
model = train_model(X_train, Y_train, **params)

-------------- training model -----------------
Device: cuda:0
epoch:0/5, train loss:1.92983877658844, train acc:0.4077539424316138, train F1:0.04053816199302673, test acc:0.3955, test F1:0.03986789286136627
epoch:1/5, train loss:1.3237147331237793, train acc:0.4399939543594991, train F1:0.04280932620167732, test acc:0.42082, test F1:0.042638279497623444
epoch:2/5, train loss:0.7139589190483093, train acc:0.4632533307190023, train F1:0.04432101920247078, test acc:0.43648, test F1:0.04369222745299339
epoch:3/5, train loss:0.3186221122741699, train acc:0.48211736303866964, train F1:0.04540267586708069, test acc:0.4479, test F1:0.04550362378358841
epoch:4/5, train loss:0.10799559950828552, train acc:0.4980361880602685, train F1:0.046328503638505936, test acc:0.45524, test F1:0.046244703233242035
-------------- training model finished-----------------


In [28]:
torch.save(model, GOOGLE_DRIVE_PATH + '/model_textCNN.pt')